# HOMEWORK 2.2: Ensemble Learning

# Description of HW2.2

The aim of this assignment is to use Ensemble Learning to solve a problem. In this way, it is aimed to understand the benefits of Ensemble Learning and to teach the usage details of different ensemble approaches with the help of ScikitLearn library. 

The following methods will be implemented within the scope of this assignment. These are:
- Voting Classifier (hard & soft voting) 


In [ ]:
# Common imports
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

In [ ]:
# to make this notebook's output stable across runs
np.random.seed(42)

In [ ]:
# generate moon dataset
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# TODO 2.2.1 Voting classifier in Scikit-Learn
- Train voting classifiers in Scikit-Learn, composed of at least three diverse classifiers


In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf2 = ExtraTreesClassifier(n_estimators=100, random_state=42)
clf3 = GradientBoostingClassifier(n_estimators=100, random_state=42)

#define your hard voting classifier
voting_clf_hard = VotingClassifier(
    estimators=[('rf', clf1), ('et', clf2), ('gb', clf3)],
    voting='hard'
)


#train your voting classifier using X_train, y_train
voting_clf_hard.fit(X_train, y_train) 



#write obtained individual classifiers. Compare them with "Voting Classifiers (hard and soft)" for "X_test/y_test" data
from sklearn.metrics import accuracy_score
for clf in (clf1, clf2, clf3, voting_clf_hard):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))


# TODO 2.2.2 Bagging/Pasting
- One way to get a diverse set of classifiers is to use very different training algorithms
- Another approach is to use the same training algorithm for every predictor, but to train them on different random subsets of the training set
    - When sampling is performed with replacement, this method is called ***bagging*** 
        - short for ***bootstrap aggregating***
    - When sampling is performed without replacement, it is called pasting
- Both bagging and pasting allow training instances to be sampled several times across multiple predictors
- Only bagging allows training instances to be sampled several times for the same predictor
- Predictors can all be trained in parallel, via different CPU cores or even different servers.
- Similarly, predictions can be made in parallel.
- This is one of the reasons why bagging and pasting are such popular methods: they scale very well.

In [ ]:
# Scikit-Learn offers a simple API for both bagging and pasting with the BaggingClassifier class 
#  
#   (this is an example of bagging, but if you want to use pasting instead, just set bootstrap=False). 
# The n_jobs parameter tells Scikit-Learn the number of CPU cores to use for training and predictions 
#   (–1 tells Scikit-Learn to use all available cores):

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


# Write Bagging Classifier and train it using ensemble of 500 Decision Tree classifiers, 
#  each trained on 100 training instances randomly sampled from the training set with replacement 

bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)
bag_clf.fit(X_train, y_train) 



# bagging accuracy score
# Using X_test dataset calculate/print Bagging Accuracy Score (using accuracy_score metric)
from sklearn.metrics import accuracy_score

y_pred_bag = bag_clf.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred_bag)
print(f"Bagging Accuracy Score: {bagging_accuracy}")


In [ ]:
# Without bagging accuracy score
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)


# Calculate "Tree Classifier" accuracy score
# Using X_test dataset calculate/print Tree Classifier Accuracy Score (using accuracy_score metric)
y_pred_tree = tree_clf.predict(X_test)
print("Tree Classifier Accuracy Score:", accuracy_score(y_test, y_pred_tree))




# TODO 2.2.2 Out-of-Bag evaluation
- With bagging, some instances may be sampled several times for any given predictor, 
 while others may not be sampled at all. 
- By default a BaggingClassifier samples m training instances with replacement (bootstrap=True), 
 where m is the size of the training set
- Only about 60% of the training instances are sampled on average for each predictor
- The remaining 40% of the training instances that are not sampled are called out-of-bag (oob) instances
- Since a predictor never sees the oob instances during training, it can be evaluated on these instances, 
without the need for a separate validation set or cross-validation
- In Scikit-Learn, you can set oob_score=True when creating a BaggingClassifier
 to request an automatic oob evaluation after training

In [ ]:
#write your out-of-bag (oob) classifier and train it. 
oob_bag_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=500,
    max_samples=100,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=42
)

# Train the out-of-bag (oob) Bagging Classifier
oob_bag_clf.fit(X_train, y_train)

# According to this oob evaluation print your oob score for test dataset 
print("Out-of-Bag (oob) Score:", oob_bag_clf.oob_score_)



# Calculate oob bagging accuracy score
from sklearn.metrics import accuracy_score

y_pred_oob = oob_bag_clf.predict(X_test)
print("Out-of-Bag (oob) Bagging Accuracy Score:", accuracy_score(y_test, y_pred_oob))

# TODO 2.2.4 Random Forests
- Random Forest is an ensemble of Decision Trees
- Generally trained via the bagging method typically with max_samples set to the size of the training set
- Instead of building a BaggingClassifier and passing it a DecisionTreeClassifier, 
you can instead use the RandomForestClassifier class, which is more convenient and optimized for Decision Trees

- Train a Random Forest classifier with 500 trees (each limited to maximum 16 nodes), using all available CPU cores:

In [ ]:
# RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Write your RandomForestClassifier classifier using sklearn RandomForestClassifier class and train it. 
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1, random_state=42)

# Train the Random Forest Classifier
rnd_clf.fit(X_train, y_train)

# Calculate/print the prediction accuracy of the rnd_clf for Test Data
y_pred_rnd = rnd_clf.predict(X_test)
print("Random Forest Classifier Accuracy Score:", accuracy_score(y_test, y_pred_rnd))

# Write your RandomForestClassifier classifier using BaggingClassifier equivalent and train it.(estimator=500, leafnode=16) 
bag_rnd_clf = BaggingClassifier(
    base_estimator=DecisionTreeClassifier(max_leaf_nodes=16),
    n_estimators=500,
    n_jobs=-1,
    random_state=42
)

# Train the Random Forest Classifier using BaggingClassifier
bag_rnd_clf.fit(X_train, y_train)

# Calculate/print the prediction accuracy of the bag_rnd_clf for Test Data
y_pred_bag_rnd = bag_rnd_clf.predict(X_test)
print("Random Forest Classifier (BaggingClassifier) Accuracy Score:", accuracy_score(y_test, y_pred_bag_rnd))


# TODO 2.2.5 Write Your Novel Ensemble Model 
- Write your customized Ensemble Classifier written by yourself 
- Try to get the highest score for the same dataset
- There no limit. There is no specific constraints. Note that the classifier you write should be an Ensemble Classifier. 

Note: Students with the highest score on the assignment will be awarded an additional 10 points as an assignment score. All submitted scores will be ranked in descending order and the top 5 students will be awarded an additional +10 points for WH2.2.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.ensemble import BaseEnsemble
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils import resample

class MyEnsembleClassifier(BaseEnsemble, BaseEstimator, ClassifierMixin):
    def __init__(self, base_estimator, n_estimators=3):
        self.base_estimator = base_estimator
        self.n_estimators = n_estimators
        self.estimators_ = []
        self.weights = []

    def fit(self, X, y):
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)

        # Clear any previous fit results
        self.estimators_ = []
        self.weights = []

        # Fit n_estimators base models
        for _ in range(self.n_estimators):
            # Bootstrap resample the data
            X_bootstrap, y_bootstrap = resample(X, y, random_state=np.random.randint(1, 1000))

            # Clone the base estimator and fit on the resampled data
            estimator = clone(self.base_estimator)
            estimator.fit(X_bootstrap, y_bootstrap)

            # Append the trained base estimator to the list of estimators
            self.estimators_.append(estimator)

            # Assign equal weight to each base model (you can customize weights based on performance)
            self.weights.append(1.0 / self.n_estimators)

        return self

    def predict(self, X):
        # Check if the classifier has been fitted
        check_is_fitted(self)

        # Make predictions from all base models
        predictions = np.array([estimator.predict(X) for estimator in self.estimators_])

        # Apply the weighted majority voting
        final_predictions = np.average(predictions, axis=0, weights=self.weights)

        # Convert probabilities to binary predictions
        binary_predictions = (final_predictions >= 0.5).astype(int)

        return binary_predictions

    
# Write your Ensemble Classifier and train it. 

my_base_estimator = DecisionTreeClassifier(max_depth=3, random_state=42)
my_ensemble_clf = MyEnsembleClassifier(base_estimator=my_base_estimator, n_estimators=5)

# Train your custom ensemble classifier
my_ensemble_clf.fit(X_train, y_train)

# Calculate accuracy score
from sklearn.metrics import accuracy_score

y_pred_ensemble = my_ensemble_clf.predict(X_test)
accuracy_ensemble = accuracy_score(y_test, y_pred_ensemble)
print("Custom Ensemble Classifier Accuracy Score:", accuracy_ensemble)
